# 2016 Election Data
### Load the data excel file

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import psycopg2
from sqlalchemy import create_engine

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

In [2]:
"""
Extract multiple sheets from the Excel file and load them into Pandas data frames. 
A dictionary allows you to store each data frame under a unique key (the sheet name), 
making it easy to access and work with each data frame later.
"""
# Define the file path to the Excel file
file_path = 'Election_Results/federalelections2016.xlsx'

# Define a list of sheet names to extract
sheet_names = ['2016 Pres General Results', 
               '2016 US Senate Results by State', 
               '2016 US House Results by State']

# Create an empty dictionary to store the data frames
data_frames = {}

# Loop over the sheet names and read them into data frames
for name in sheet_names:
    data_frames[name] = pd.read_excel(file_path, sheet_name=name)

# Print the data frames to verify that they were loaded correctly
# for name, df in data_frames.items():
#     print(f"{name}:")
#     print(df.head())


2016 Pres Primary Results
2016 US House Results by State
2016 Pres Primary Results
2016 US House Results by State
2016 Pres Primary Results
2016 US House Results by State


In [3]:
df_president_2016 = data_frames['2016 Pres General Results']
df_president_2016.head()
# 681 rows 18 columns

1     FEC ID    STATE STATE ABBREVIATION GENERAL ELECTION DATE FIRST NAME   
0  2  P80001571  Alabama                 AL            2016-11-08  Donald J.  \
1  3  P00003392  Alabama                 AL            2016-11-08    Hillary   
2  4  P20002671  Alabama                 AL            2016-11-08       Gary   
3  5        NaN  Alabama                 AL            2016-11-08        NaN   
4  6  P20003984  Alabama                 AL            2016-11-08       Jill   

   LAST NAME LAST NAME,  FIRST TOTAL VOTES PARTY  GENERAL RESULTS   
0      Trump  Trump, Donald J.         NaN   REP        1318255.0  \
1    Clinton  Clinton, Hillary         NaN   DEM         729547.0   
2    Johnson     Johnson, Gary         NaN   IND          44467.0   
3  Scattered       Scattered,          NaN     W          21712.0   
4      Stein       Stein, Jill         NaN   IND           9391.0   

   TOTAL VOTES #  GENERAL % WINNER INDICATOR  
0            NaN   0.620831                W  
1            NaN   0.343579              NaN  
2            NaN   0.020942              NaN  
3            NaN   0.010225              NaN  
4            NaN   0.004423              NaN

In [4]:
df_selection = df_president_2016[['STATE ABBREVIATION', 'LAST NAME', 'PARTY', 'GENERAL %', 'GENERAL RESULTS']]
df_pres_2016 = df_selection.rename(columns={
    'STATE ABBREVIATION': 'State',
    'LAST NAME': 'Last Name',
    'PARTY': 'Party',
    'GENERAL %': 'Vote %',
    'GENERAL RESULTS': 'Vote count'
})
df_pres_2016.head()

State  Last Name Party    Vote %  Vote count
0    AL      Trump   REP  0.620831   1318255.0
1    AL    Clinton   DEM  0.343579    729547.0
2    AL    Johnson   IND  0.020942     44467.0
3    AL  Scattered     W  0.010225     21712.0
4    AL      Stein   IND  0.004423      9391.0

In [5]:
# add year and office columns
df_pres_2016['Year'] = 2016
df_pres_2016['Office'] = 'Pres'

# Move the Year and Office columns to the front
new_cols = ['Year', 'Office', 'State', 'Last Name', 'Party', 'Vote %', 'Vote count']
df_pres_2016 = df_pres_2016.reindex(columns=new_cols)

# Include only Candidates R, D, IND (independent)
df_pres_2016 = df_pres_2016[df_pres_2016['Party'].isin(['REP', 'DEM', 'IND'])]

df_pres_2016.tail()

Year Office State     Last Name Party    Vote %  Vote count
910  2016   Pres    WI       Clinton   DEM  0.464538   1382536.0
928  2016   Pres    WY         Trump   REP  0.681726    174419.0
929  2016   Pres    WY       Clinton   DEM  0.218774     55973.0
932  2016   Pres    WY         Stein   IND  0.009830      2515.0
934  2016   Pres    WY  De La Fuente   IND  0.002771       709.0

In [6]:
# Remove any unseen NaN's from num rows.
df_p_2016 = df_pres_2016.dropna(subset=['Vote %'])
df_p_2016 = df_p_2016.dropna(subset=['Vote count'])
# confirms NaN's are gone
print(df_p_2016[df_p_2016['Vote %'].isna()])
print(df_p_2016[df_p_2016['Vote count'].isna()])

Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []
Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []


In [7]:
# convert Vote count to float for math
df_p_2016['Vote count'] = df_p_2016['Vote count'].astype(int)
df_p_2016.dtypes

Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count      int32
dtype: object

In [8]:
df_p_2016.shape
# gives 120 rows.

(120, 7)

In [9]:
df_p_2016.head()

Year Office State Last Name Party    Vote %  Vote count
0  2016   Pres    AL     Trump   REP  0.620831     1318255
1  2016   Pres    AL   Clinton   DEM  0.343579      729547
2  2016   Pres    AL   Johnson   IND  0.020942       44467
4  2016   Pres    AL     Stein   IND  0.004423        9391
7  2016   Pres    AK     Trump   REP  0.512815      163387

# 2016 Senate data

In [10]:
df_senate_2016 = data_frames['2016 US Senate Results by State']
df_senate_2016.head()
# 681 rows 18 columns

1 STATE ABBREVIATION    STATE    D    FEC ID#  (I) CANDIDATE NAME (First)   
0  2                NaN      NaN  NaN        NaN  NaN                    NaN  \
1  3                 AL  Alabama    S  S6AL00013  (I)             Richard C.   
2  4                 AL  Alabama    S  S6AL00310  NaN               Jonathan   
3  5                 AL  Alabama    S  S6AL00328  NaN                   John   
4  6                 AL  Alabama    S  S6AL00336  NaN                 Marcus   

  CANDIDATE NAME (Last)       CANDIDATE NAME TOTAL VOTES  ... RUNOFF VOTES   
0                   NaN                  NaN         NaN  ...          NaN  \
1                Shelby   Shelby, Richard C.         NaN  ...          NaN   
2             McConnell  McConnell, Jonathan         NaN  ...          NaN   
3                Martin         Martin, John         NaN  ...          NaN   
4                Bowman       Bowman, Marcus         NaN  ...          NaN   

  RUNOFF %  GENERAL VOTES   GENERAL %  GE RUNOFF ELECTION VOTES (LA)   
0      NaN             NaN        NaN                            NaN  \
1      NaN       1335104.0   0.639588                            NaN   
2      NaN             NaN        NaN                            NaN   
3      NaN             NaN        NaN                            NaN   
4      NaN             NaN        NaN                            NaN   

   GE RUNOFF ELECTION % (LA)  COMBINED GE PARTY TOTALS (CT, NY, SC)   
0                        NaN                                    NaN  \
1                        NaN                                    NaN   
2                        NaN                                    NaN   
3                        NaN                                    NaN   
4                        NaN                                    NaN   

   COMBINED % (CT, NY, SC)  GE WINNER INDICATOR  FOOTNOTES  
0                      NaN                  NaN        NaN  
1                      NaN                    W        NaN  
2                      NaN                  NaN        NaN  
3                      NaN                  NaN        NaN  
4                      NaN                  NaN        NaN  

[5 rows x 23 columns]

In [11]:
df_selection = df_senate_2016[['STATE ABBREVIATION', 'CANDIDATE NAME (Last)', 'PARTY', 'GENERAL %', 'GENERAL VOTES ']]

df_sen_2016 = df_selection.rename(columns={
    'STATE ABBREVIATION': 'State',
    'CANDIDATE NAME (Last)': 'Last Name',
    'PARTY': 'Party',
    'GENERAL %': 'Vote %',
    'GENERAL VOTES ': 'Vote count'
})
df_sen_2016.head()

State  Last Name Party    Vote %  Vote count
0   NaN        NaN   NaN       NaN         NaN
1    AL     Shelby     R  0.639588   1335104.0
2    AL  McConnell     R       NaN         NaN
3    AL     Martin     R       NaN         NaN
4    AL     Bowman     R       NaN         NaN

In [12]:
# add year and office columns
df_sen_2016['Year'] = 2016
df_sen_2016['Office'] = 'Senate'

# Move the Year and Office columns to the front
new_cols = ['Year', 'Office', 'State', 'Last Name', 'Party', 'Vote %', 'Vote count']
df_sen_2016 = df_sen_2016.reindex(columns=new_cols)

# Include only Candidates R, D, IND (independent)
df_sen_2016 = df_sen_2016[df_sen_2016['Party'].isin(['R', 'D', 'IND'])]
df_sen_2016['Party'] = df_sen_2016['Party'].replace('R', 'REP')
df_sen_2016['Party'] = df_sen_2016['Party'].replace('D', 'DEM')

df_sen_2016.head()
# give 276 rows and 7 columns

Year  Office State  Last Name Party    Vote %  Vote count
1  2016  Senate    AL     Shelby   REP  0.639588   1335104.0
2  2016  Senate    AL  McConnell   REP       NaN         NaN
3  2016  Senate    AL     Martin   REP       NaN         NaN
4  2016  Senate    AL     Bowman   REP       NaN         NaN
5  2016  Senate    AL     McGill   REP       NaN         NaN

In [13]:
# Remove any unseen NaN's from num rows.
df_s_2016 = df_sen_2016.dropna(subset=['Vote %'])
df_s_2016 = df_s_2016.dropna(subset=['Vote count'])
# confirms NaN's are gone
print(df_s_2016[df_s_2016['Vote %'].isna()])
print(df_s_2016[df_s_2016['Vote count'].isna()])

Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []
Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []


In [14]:
# convert Vote count to float for math
df_s_2016['Vote count'] = df_s_2016['Vote count'].astype(int)
df_s_2016.dtypes

Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count      int32
dtype: object

In [15]:
df_s_2016.head()
# gives 84 rows

Year  Office State  Last Name Party    Vote %  Vote count
1   2016  Senate    AL     Shelby   REP  0.639588     1335104
7   2016  Senate    AL   Crumpton   DEM  0.358673      748709
14  2016  Senate    AK  Murkowski   REP  0.443580      138149
22  2016  Senate    AK   Metcalfe   DEM  0.116234       36200
31  2016  Senate    AZ     McCain   REP  0.537105     1359267

# 2016 House data

In [16]:
df_house_2016 = data_frames['2016 US House Results by State']
df_house_2016.head()
# 4131 rows 23 columns

1 STATE ABBREVIATION    STATE    D    FEC ID#  (I) CANDIDATE NAME (First)   
0  2                NaN      NaN  NaN        NaN  NaN                    NaN  \
1  3                 AL  Alabama  NaN        NaN  NaN                    NaN   
2  4                 AL  Alabama   01  H4AL01123  (I)                Bradley   
3  5                 AL  Alabama   01  H6AL01060  NaN                   Dean   
4  6                 AL  Alabama   01        NaN  NaN                    NaN   

  CANDIDATE NAME (Last)  CANDIDATE NAME   TOTAL VOTES  ... RUNOFF VOTES   
0                   NaN             NaN           NaN  ...          NaN  \
1                   NaN             NaN           NaN  ...          NaN   
2                 Byrne  Byrne, Bradley           NaN  ...          NaN   
3                 Young     Young, Dean           NaN  ...          NaN   
4                   NaN             NaN  Party Votes:  ...          NaN   

  RUNOFF %  GENERAL VOTES   GENERAL %  GE RUNOFF ELECTION VOTES (LA)   
0      NaN             NaN        NaN                            NaN  \
1      NaN             NaN        NaN                            NaN   
2      NaN          208083   0.963825                            NaN   
3      NaN             NaN        NaN                            NaN   
4      NaN             NaN        NaN                            NaN   

  GE RUNOFF ELECTION % (LA)  COMBINED GE PARTY TOTALS (CT, NY, SC)   
0                       NaN                                    NaN  \
1                       NaN                                    NaN   
2                       NaN                                    NaN   
3                       NaN                                    NaN   
4                       NaN                                    NaN   

   COMBINED % (CT, NY, SC)  GE WINNER INDICATOR  FOOTNOTES  
0                      NaN                  NaN        NaN  
1                      NaN                  NaN        NaN  
2                      NaN                    W        NaN  
3                      NaN                  NaN        NaN  
4                      NaN                  NaN        NaN  

[5 rows x 23 columns]

In [17]:
df_selection = df_house_2016[['STATE ABBREVIATION', 'CANDIDATE NAME (Last)', 'PARTY', 'GENERAL %', 'GENERAL VOTES ']]

df_hou_2016 = df_selection.rename(columns={
    'STATE ABBREVIATION': 'State',
    'CANDIDATE NAME (Last)': 'Last Name',
    'PARTY': 'Party',
    'GENERAL %': 'Vote %',
    'GENERAL VOTES ': 'Vote count'
})
df_hou_2016.head()
# gives 4131 rows

State Last Name Party    Vote % Vote count
0   NaN       NaN   NaN       NaN        NaN
1    AL       NaN   NaN       NaN        NaN
2    AL     Byrne     R  0.963825     208083
3    AL     Young     R       NaN        NaN
4    AL       NaN     R       NaN        NaN

In [18]:
# add year and office columns
df_hou_2016['Year'] = 2016
df_hou_2016['Office'] = 'House'

# Move the Year and Office columns to the front
new_cols = ['Year', 'Office', 'State', 'Last Name', 'Party', 'Vote %', 'Vote count']
df_hou_2016 = df_hou_2016.reindex(columns=new_cols)

# Include only Candidates R, D, IND (independent)
df_hou_2016 = df_hou_2016[df_hou_2016['Party'].isin(['R', 'D', 'IND'])]
df_hou_2016['Party'] = df_hou_2016['Party'].replace('R', 'REP')
df_hou_2016['Party'] = df_hou_2016['Party'].replace('D', 'DEM')

df_hou_2016.head()
# 2096 rows 7 columns

Year Office State  Last Name Party    Vote % Vote count
2  2016  House    AL      Byrne   REP  0.963825     208083
3  2016  House    AL      Young   REP       NaN        NaN
4  2016  House    AL        NaN   REP       NaN        NaN
8  2016  House    AL       Roby   REP  0.487685     134886
9  2016  House    AL  Gerritson   REP       NaN        NaN

In [19]:
# Remove any unseen NaN's from num rows.
df_h_2016 = df_hou_2016.dropna(subset=['Vote %'])
df_h_2016 = df_h_2016.dropna(subset=['Vote count'])
# confirms NaN's are gone
print(df_h_2016[df_h_2016['Vote %'].isna()])
print(df_h_2016[df_h_2016['Vote count'].isna()])

Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []
Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []


In [20]:
df_h_2016.dtypes

Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count     object
dtype: object

In [21]:
# convert Vote count to float for math
df_h_2016['Vote count'] = df_h_2016['Vote count'].astype(int)
df_h_2016.dtypes

Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count      int32
dtype: object

In [22]:
df_h_2016.head()
#gives 848 rows

Year Office State Last Name Party    Vote %  Vote count
2   2016  House    AL     Byrne   REP  0.963825      208083
8   2016  House    AL      Roby   REP  0.487685      134886
12  2016  House    AL    Mathis   DEM  0.405262      112089
16  2016  House    AL    Rogers   REP  0.669318      192164
19  2016  House    AL     Smith   DEM  0.329320       94549

# Add to Database

In [115]:
# in postgreSQL created database called: electiondb
# create table in the postgresSQL first then ran code below
# CREATE TABLE results (
#     year INTEGER NOT NULL,
#     office VARCHAR(50) NOT NULL,
#     state VARCHAR(50) NOT NULL,
#     last_name VARCHAR(50) NOT NULL,
#     party VARCHAR(50) NOT NULL,
#     vote_percentage NUMERIC(5, 2) NOT NULL,
#     vote_count BIGINT NOT NULL,
#     PRIMARY KEY (year, office, state, last_name, party)
# );

# run this line in postgresSQL
# GRANT ALL PRIVILEGES ON DATABASE electiondb TO postgres;

In [23]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database
df_p_2016.to_sql("results", engine, if_exists="append", index=False)
print("Presidential table loaded successfully")
df_s_2016.to_sql("results", engine, if_exists="append", index=False)
df_h_2016.to_sql("results", engine, if_exists="append", index=False)

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


Presidential table loaded successfully


In [24]:
# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM results", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()
# gives total 3136 rows for 2020-2016

      Year Office State  Last Name Party    Vote %  Vote count
0     2020   Pres    AL      Trump   REP  0.620316     1441170
1     2020   Pres    AL      Biden   DEM  0.365700      849624
2     2020   Pres    AL  Jorgensen   IND  0.010836       25176
3     2020   Pres    AK      Trump   REP  0.528331      189951
4     2020   Pres    AK      Biden   DEM  0.427720      153778
...    ...    ...   ...        ...   ...       ...         ...
3131  2016  House    WI      Hoeft   DEM  0.382705      138643
3132  2016  House    WI  Gallagher   REP  0.626456      227892
3133  2016  House    WI     Nelson   DEM  0.372978      135682
3134  2016  House    WY     Cheney   REP  0.620297      156176
3135  2016  House    WY     Greene   DEM  0.299735       75466

[3136 rows x 7 columns]


In [ ]:
# note we will need to update the party column identifiers to be the same

# UPDATE results
# SET party = 'R'
# WHERE party = 'REP';

# UPDATE results
# SET party = 'D'
# WHERE party = 'DEM';

# query the data base
# SELECT * FROM results;
# SELECT * FROM results LIMIT 5;
# SELECT column_name
#     FROM information_schema.columns
#     WHERE table_name = 'results';
# SELECT 'Last Name' FROM results WHERE 'State'='CA';
